In [13]:
import numpy as np
import os
from progressbar import ProgressBar

In [14]:
pileup_paths = [
    './e-coli-NCTC86-racon-MSA/pileups-X-0.npy',
    './k-pneumoniae-NCTC204-racon-MSA/pileups-X-0.npy',
    './m-morgani-NCTC235-racon-MSA/pileups-X-0.npy',
    './s-enterica-NCTC92-racon-MSA/pileups-X-0.npy',
    './s-enterica-NCTC129-racon-MSA/pileups-X-0.npy',
]
labels_paths = [
    './e-coli-NCTC86-racon-MSA/pileups-y-0.npy',
    './k-pneumoniae-NCTC204-racon-MSA/pileups-y-0.npy',
    './m-morgani-NCTC235-racon-MSA/pileups-y-0.npy',
    './s-enterica-NCTC92-racon-MSA/pileups-y-0.npy',
    './s-enterica-NCTC129-racon-MSA/pileups-y-0.npy',
]

In [77]:
def convert_label_to_name(label):
    mapping = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: 'I', 5: 'D'}
    return mapping[np.argmax(label)]

def print_deletions_neighbourhood(labels_path, neighbourhood_size):
    labels = np.load(labels_path)
    print(labels.shape)

    print('A:', np.sum(labels[:, 0]))
    print('C:', np.sum(labels[:, 1]))
    print('G:', np.sum(labels[:, 2]))
    print('T', np.sum(labels[:, 3]))
    print('I:', np.sum(labels[:, 4]))
    print('D:', np.sum(labels[:, 5]))

    deletion_indices = np.where(np.all(labels == [0, 0, 0, 0, 0, 1, 0], axis=1))[0]

    for deletion_index in deletion_indices:
        classes = [convert_label_to_name(label) for label in labels[
            deletion_index-neighbourhood_size:deletion_index+neighbourhood_size+1]]
        classes = ''.join(classes)
        classes = classes[:neighbourhood_size] + '-D-' + classes[neighbourhood_size+1:]

        print(classes)
        
def count_homopolymers_with_deletion(labels_path, print_flag=False):
    labels = np.load(labels_path)

    deletion_indices = np.where(np.all(labels == [0, 0, 0, 0, 0, 1, 0], axis=1))[0]
    
     # Homopolymers before a deletion.
    homopolymer_count_before = 0
    insertion_homopolymer_count_before = 0
    for deletion_index in deletion_indices:
        if np.argmax(labels[deletion_index - 1]) == 5: # deletion, we skip it
            continue

        if np.all(labels[deletion_index - 1] == labels[deletion_index - 2]): # two subsequent labels are the same
            if np.argmax(labels[deletion_index - 1]) == 4: # insertion, not counted
                insertion_homopolymer_count_before += 1
            
            if print_flag:
                classes = [convert_label_to_name(label) for label in labels[
                    deletion_index-2:deletion_index+1]]
                classes = ''.join(classes)
                print(classes)
            homopolymer_count_before += 1

    # Homopolymers after a deletion.
    homopolymer_count_after = 0
    insertion_homopolymer_count_after = 0
    for deletion_index in deletion_indices:
        if np.argmax(labels[deletion_index + 1]) == 5: # deletion, we skip it
            continue

        if np.all(labels[deletion_index + 1] == labels[deletion_index + 2]): # two subsequent labels are the same
            if np.argmax(labels[deletion_index + 1]) == 4: # insertion, not counted
                insertion_homopolymer_count_after += 1
            
            if print_flag:
                classes = [convert_label_to_name(label) for label in labels[
                    deletion_index:deletion_index+3]]
                classes = ''.join(classes)
                print(classes)
            homopolymer_count_after += 1

    num_deletions = deletion_indices.shape[0]
    print('For {} labels:'.format(labels_path))
    print('\tTotal deletions: {}'.format(num_deletions))
    print('\thomopolymers before deletions: {}, insertions homopolymers before deletions: {}'.format(
        homopolymer_count_before, insertion_homopolymer_count_before))
    print('\thomopolymers after deletions: {}, insertions homopolymers afters deletions: {}\n'.format(
        homopolymer_count_after, insertion_homopolymer_count_after))
    return num_deletions, homopolymer_count_before, insertion_homopolymer_count_before, \
        homopolymer_count_after, insertion_homopolymer_count_after

def count_deletion_burst_length(labels_path, print_flag=False):
    labels = np.load(labels_path)

    deletion_indices = np.where(np.all(labels == [0, 0, 0, 0, 0, 1, 0], axis=1))[0]

    deletion_burst_lengths = dict()

    last_index, length = -1, 0
    deletion_indices = np.append(deletion_indices, -1)
    for deletion_index in deletion_indices:
        if deletion_index == last_index + 1:
            length += 1
            last_index = deletion_index
        else:
            if length not in deletion_burst_lengths:
                deletion_burst_lengths[length] = 0
            deletion_burst_lengths[length] += 1 
            length = 1
            last_index = deletion_index

    if print_flag:
        print('For {} labels:'.format(labels_path))
        for length in deletion_burst_lengths:
            print('Length: {}, count: {}'.format(length, deletion_burst_lengths[length]))
    
    return deletion_burst_lengths

def add_dicts(dict_a, dict_b):
    dict_c = dict_a.copy()
    for k in dict_b:
        if k not in dict_c:
            dict_c[k] = 0
        dict_c[k] += dict_b[k]
    return dict_c

## E. coli

In [18]:
print_deletions_neighbourhood(labels_paths[0], neighbourhood_size=20)

(8429809, 5)
(8429809, 7)
[36  0  0  0  0] [1. 0. 0. 0. 0. 0. 0.]
A: 1091030.0
C: 1134011.0
G: 1130983.0
T 1088795.0
I: 3972157.0
D: 12833.0
AIGITIGICGGIGIIIIICI-D-TTTTTTTTCIIGAIICCAAI
ITIICAGICIIIIGITTTIC-D-GIIGIITCICIGGGIIGATI
IGAIITIGTIICAICGICGI-D-ICICIGITIAITITTCCGIT
IAAIGIAGITITICTIAIII-D-CITGIIGIAIAICITIIIII
TIIAIICIGITIGTCIITGC-D-GIITIIGIIIIITTGCICII
IICTTCICIIAGCCAIGIGC-D-IIIIIIGIIITGIGIIICIG
IATGITGIIGICGCIGITII-D-AAIIICIIIITAITTIGATI
GICIICCIIIGITGIAIITG-D-IIIAIGIGAAIIAIAGIIIT
CIGIGIIIAITIATIGGIII-D-GIICGIGGICAAITIIGAIC
GIAITCIGIGIITTCGIGGI-D-ITTAGGIDTICIICAIIIGT
IGIITTCGIGGIDITTAGGI-D-TICIICAIIIGTIGCCITGI
CAIGCIIIAITIAIITAGGI-D-IGGCICTICGGGIITIITIG
ICIGTIITCTIICIICAGIC-D-IAIAGCGIIIAIICITGIIG
AGGIIIICIAAATTIAITTT-D-AAIICGIICAAIGGIGIACI
GAIIAATGCITIGCITGGIA-D-ICIAIATTAAIGIITCAIGI
TICIITIACIGIAIIITITI-D-TTIIIAAGIICICAIIICGI
ITIIGCIICIIIGITAIGIG-D-GIATIIGAITTIITICCGCI
GCIIGIACITIIGIIGTIIC-D-CAIICCIICIIGIITIACIC
CIGAAIGCICAIGIITITIC-D-CTIIGGIIITIIIGCIGTTI
IACCICIAIICITCITTIIC-D-

DDGTDDDDDDGDDADDDDDD-D-DDDDADTTDDDDDDDDCDTA
DGTDDDDDDGDDADDDDDDD-D-DDDADTTDDDDDDDDCDTAG
GTDDDDDDGDDADDDDDDDD-D-DDADTTDDDDDDDDCDTAGD
TDDDDDDGDDADDDDDDDDD-D-DADTTDDDDDDDDCDTAGDD
DDDDDDGDDADDDDDDDDDD-D-ADTTDDDDDDDDCDTAGDDD
DDDDGDDADDDDDDDDDDDA-D-TTDDDDDDDDCDTAGDDDDD
DGDDADDDDDDDDDDDADTT-D-DDDDDDDCDTAGDDDDDDDG
GDDADDDDDDDDDDDADTTD-D-DDDDDDCDTAGDDDDDDDGD
DDADDDDDDDDDDDADTTDD-D-DDDDDCDTAGDDDDDDDGDD
DADDDDDDDDDDDADTTDDD-D-DDDDCDTAGDDDDDDDGDDD
ADDDDDDDDDDDADTTDDDD-D-DDDCDTAGDDDDDDDGDDDG
DDDDDDDDDDDADTTDDDDD-D-DDCDTAGDDDDDDDGDDDGD
DDDDDDDDDDADTTDDDDDD-D-DCDTAGDDDDDDDGDDDGDD
DDDDDDDDDADTTDDDDDDD-D-CDTAGDDDDDDDGDDDGDDC
DDDDDDDADTTDDDDDDDDC-D-TAGDDDDDDDGDDDGDDCDD
DDDADTTDDDDDDDDCDTAG-D-DDDDDDGDDDGDDCDDDTDD
DDADTTDDDDDDDDCDTAGD-D-DDDDDGDDDGDDCDDDTDDD
DADTTDDDDDDDDCDTAGDD-D-DDDDGDDDGDDCDDDTDDDD
ADTTDDDDDDDDCDTAGDDD-D-DDDGDDDGDDCDDDTDDDDD
DTTDDDDDDDDCDTAGDDDD-D-DDGDDDGDDCDDDTDDDDDD
TTDDDDDDDDCDTAGDDDDD-D-DGDDDGDDCDDDTDDDDDDD
TDDDDDDDDCDTAGDDDDDD-D-GDDDGDDCDDDTDDDDDDDG
DDDDDDDCDTAGDDDDDDDG-D-DDGDDCDDD

TITGAAITIGAITDDDDADD-D-DDDAAAIDIACIAGCGIDCD
ITGAAITIGAITDDDDADDD-D-DDAAAIDIACIAGCGIDCDD
TGAAITIGAITDDDDADDDD-D-DAAAIDIACIAGCGIDCDDD
GAAITIGAITDDDDADDDDD-D-AAAIDIACIAGCGIDCDDDT
IGAITDDDDADDDDDDAAAI-D-IACIAGCGIDCDDDTTTDTD
DDDDDDAAAIDIACIAGCGI-D-CDDDTTTDTDDDTIITIDDD
DDDDAAAIDIACIAGCGIDC-D-DDTTTDTDDDTIITIDDDDD
DDDAAAIDIACIAGCGIDCD-D-DTTTDTDDDTIITIDDDDDD
DDAAAIDIACIAGCGIDCDD-D-TTTDTDDDTIITIDDDDDDA
AIDIACIAGCGIDCDDDTTT-D-TDDDTIITIDDDDDDATTDC
DIACIAGCGIDCDDDTTTDT-D-DDTIITIDDDDDDATTDCDD
IACIAGCGIDCDDDTTTDTD-D-DTIITIDDDDDDATTDCDDD
ACIAGCGIDCDDDTTTDTDD-D-TIITIDDDDDDATTDCDDDT
GIDCDDDTTTDTDDDTIITI-D-DDDDDATTDCDDDTITIATD
IDCDDDTTTDTDDDTIITID-D-DDDDATTDCDDDTITIATDD
DCDDDTTTDTDDDTIITIDD-D-DDDATTDCDDDTITIATDDT
CDDDTTTDTDDDTIITIDDD-D-DDATTDCDDDTITIATDDTC
DDDTTTDTDDDTIITIDDDD-D-DATTDCDDDTITIATDDTCD
DDTTTDTDDDTIITIDDDDD-D-ATTDCDDDTITIATDDTCDD
TDTDDDTIITIDDDDDDATT-D-CDDDTITIATDDTCDDADDI
TDDDTIITIDDDDDDATTDC-D-DDTITIATDDTCDDADDIDD
DDDTIITIDDDDDDATTDCD-D-DTITIATDDTCDDADDIDDI
DDTIITIDDDDDDATTDCDD-D-TITIATDDT

GDDGDDDDCDDCDDDDDDTD-D-DDDIADDDCDDIADIDDDII
DDGDDDDCDDCDDDDDDTDD-D-DDIADDDCDDIADIDDDIIA
DGDDDDCDDCDDDDDDTDDD-D-DIADDDCDDIADIDDDIIAD
GDDDDCDDCDDDDDDTDDDD-D-IADDDCDDIADIDDDIIADI
DDCDDCDDDDDDTDDDDDIA-D-DDCDDIADIDDDIIADIDDD
DCDDCDDDDDDTDDDDDIAD-D-DCDDIADIDDDIIADIDDDI
CDDCDDDDDDTDDDDDIADD-D-CDDIADIDDDIIADIDDDIT
DCDDDDDDTDDDDDIADDDC-D-DIADIDDDIIADIDDDITID
CDDDDDDTDDDDDIADDDCD-D-IADIDDDIIADIDDDITIDI
DDDDTDDDDDIADDDCDDIA-D-IDDDIIADIDDDITIDIDDI
DDTDDDDDIADDDCDDIADI-D-DDIIADIDDDITIDIDDIGD
DTDDDDDIADDDCDDIADID-D-DIIADIDDDITIDIDDIGDI
TDDDDDIADDDCDDIADIDD-D-IIADIDDDITIDIDDIGDID
DDIADDDCDDIADIDDDIIA-D-IDDDITIDIDDIGDIDDIDD
IADDDCDDIADIDDDIIADI-D-DDITIDIDDIGDIDDIDDID
ADDDCDDIADIDDDIIADID-D-DITIDIDDIGDIDDIDDIDI
DDDCDDIADIDDDIIADIDD-D-ITIDIDDIGDIDDIDDIDID
DDIADIDDDIIADIDDDITI-D-IDDIGDIDDIDDIDIDDDDI
IADIDDDIIADIDDDITIDI-D-DIGDIDDIDDIDIDDDDIGG
ADIDDDIIADIDDDITIDID-D-IGDIDDIDDIDIDDDDIGGD
DDDIIADIDDDITIDIDDIG-D-IDDIDDIDIDDDDIGGDDTD
DIIADIDDDITIDIDDIGDI-D-DIDDIDIDDDDIGGDDTDDD
IIADIDDDITIDIDDIGDID-D-IDDIDIDDD

CDDDDGDATDGDAATADDDD-D-DDTDDDCGDGGDDDTTTDTD
DDDDGDATDGDAATADDDDD-D-DTDDDCGDGGDDDTTTDTDA
DDDGDATDGDAATADDDDDD-D-TDDDCGDGGDDDTTTDTDAD
DGDATDGDAATADDDDDDDT-D-DDCGDGGDDDTTTDTDADDC
GDATDGDAATADDDDDDDTD-D-DCGDGGDDDTTTDTDADDCD
DATDGDAATADDDDDDDTDD-D-CGDGGDDDTTTDTDADDCDT
DGDAATADDDDDDDTDDDCG-D-GGDDDTTTDTDADDCDTDDG
AATADDDDDDDTDDDCGDGG-D-DDTTTDTDADDCDTDDGDDD
ATADDDDDDDTDDDCGDGGD-D-DTTTDTDADDCDTDDGDDDD
TADDDDDDDTDDDCGDGGDD-D-TTTDTDADDCDTDDGDDDDD
DDDDDTDDDCGDGGDDDTTT-D-TDADDCDTDDGDDDDDDDDI
DDDTDDDCGDGGDDDTTTDT-D-ADDCDTDDGDDDDDDDDITG
DTDDDCGDGGDDDTTTDTDA-D-DCDTDDGDDDDDDDDITGCD
TDDDCGDGGDDDTTTDTDAD-D-CDTDDGDDDDDDDDITGCDT
DDCGDGGDDDTTTDTDADDC-D-TDDGDDDDDDDDITGCDTTA
CGDGGDDDTTTDTDADDCDT-D-DGDDDDDDDDITGCDTTADC
GDGGDDDTTTDTDADDCDTD-D-GDDDDDDDDITGCDTTADCD
GGDDDTTTDTDADDCDTDDG-D-DDDDDDDITGCDTTADCDAG
GDDDTTTDTDADDCDTDDGD-D-DDDDDDITGCDTTADCDAGA
DDDTTTDTDADDCDTDDGDD-D-DDDDDITGCDTTADCDAGAD
DDTTTDTDADDCDTDDGDDD-D-DDDDITGCDTTADCDAGADD
DTTTDTDADDCDTDDGDDDD-D-DDDITGCDTTADCDAGADDD
TTTDTDADDCDTDDGDDDDD-D-DDITGCDTT

...

IICIGCICIIITCATAICTI-D-TIIITITTICICTCIGIGII
ITCAIIIIGIACICAIIGII-D-TIICACIITIGGCIIITICC
GITTCTGIIIIIIATGIGAT-D-ITGGGCIIGGIAIIGICDIT
ATDITGGGCIIGGIAIIGIC-D-ITTGACITCICIIIGIIICI
TAGITIATCIIITATITCIA-D-DIDDTICTCTIIIIICIIIC
AGITIATCIIITATITCIAD-D-IDDTICTCTIIIIICIIICI
ITIATCIIITATITCIADDI-D-DTICTCTIIIIICIIICIAI
TIATCIIITATITCIADDID-D-TICTCTIIIIICIIICIAID
DTICTCTIIIIICIIICIAI-D-DDITDAIIATIAIITTTTDG
TICTCTIIIIICIIICIAID-D-DITDAIIATIAIITTTTDGC
ICTCTIIIIICIIICIAIDD-D-ITDAIIATIAIITTTTDGCD
CTIIIIICIIICIAIDDDIT-D-AIIATIAIITTTTDGCDAII
IDDDITDAIIATIAIITTTT-D-GCDAIIITIAIIDDDDDCDI
DITDAIIATIAIITTTTDGC-D-AIIITIAIIDDDDDCDIIID
AIITTTTDGCDAIIITIAII-D-DDDDCDIIIDTIIIGDDATI
IITTTTDGCDAIIITIAIID-D-DDDCDIIIDTIIIGDDATII
ITTTTDGCDAIIITIAIIDD-D-DDCDIIIDTIIIGDDATIII
TTTTDGCDAIIITIAIIDDD-D-DCDIIIDTIIIGDDATIIID
TTTDGCDAIIITIAIIDDDD-D-CDIIIDTIIIGDDATIIIDI
TDGCDAIIITIAIIDDDDDC-D-IIIDTIIIGDDATIIIDITD
DAIIITIAIIDDDDDCDIII-D-TIIIGDDATIIIDITDTTAA
IAIIDDDDDCDIIIDTIIIG-D-DATIIIDITDTTAATIIAIA
AIIDDDDDCDIIIDTIIIGD-D-ATIIIDITD

DDDDIGDDDDDIDDDDDTDD-D-DIDDDIDDDICDDIDDDIDA
DDDIGDDDDDIDDDDDTDDD-D-IDDDIDDDICDDIDDDIDAD
DIGDDDDDIDDDDDTDDDDI-D-DDIDDDICDDIDDDIDADDI
IGDDDDDIDDDDDTDDDDID-D-DIDDDICDDIDDDIDADDID
GDDDDDIDDDDDTDDDDIDD-D-IDDDICDDIDDDIDADDIDD
DDDDIDDDDDTDDDDIDDDI-D-DDICDDIDDDIDADDIDDDD
DDDIDDDDDTDDDDIDDDID-D-DICDDIDDDIDADDIDDDDD
DDIDDDDDTDDDDIDDDIDD-D-ICDDIDDDIDADDIDDDDDD
DDDDDTDDDDIDDDIDDDIC-D-DIDDDIDADDIDDDDDDDAD
DDDDTDDDDIDDDIDDDICD-D-IDDDIDADDIDDDDDDDADD
DDTDDDDIDDDIDDDICDDI-D-DDIDADDIDDDDDDDADDDD
DTDDDDIDDDIDDDICDDID-D-DIDADDIDDDDDDDADDDDD
TDDDDIDDDIDDDICDDIDD-D-IDADDIDDDDDDDADDDDDC
DDDIDDDIDDDICDDIDDDI-D-ADDIDDDDDDDADDDDDCII
DIDDDIDDDICDDIDDDIDA-D-DIDDDDDDDADDDDDCIIGC
IDDDIDDDICDDIDDDIDAD-D-IDDDDDDDADDDDDCIIGCD
DDIDDDICDDIDDDIDADDI-D-DDDDDDADDDDDCIIGCDII
DIDDDICDDIDDDIDADDID-D-DDDDDADDDDDCIIGCDIII
IDDDICDDIDDDIDADDIDD-D-DDDDADDDDDCIIGCDIIII
DDDICDDIDDDIDADDIDDD-D-DDDADDDDDCIIGCDIIIID
DDICDDIDDDIDADDIDDDD-D-DDADDDDDCIIGCDIIIIDD
DICDDIDDDIDADDIDDDDD-D-DADDDDDCIIGCDIIIIDDD
ICDDIDDDIDADDIDDDDDD-D-ADDDDDCII

IDDDDDDDDDDTGDDDGDDD-D-DDDTDDDDDDDDDDTDTTDI
DDDDDDDDDDTGDDDGDDDD-D-DDTDDDDDDDDDDTDTTDIC
DDDDDDDDDTGDDDGDDDDD-D-DTDDDDDDDDDDTDTTDICI
DDDDDDDDTGDDDGDDDDDD-D-TDDDDDDDDDDTDTTDICID
DDDDDDTGDDDGDDDDDDDT-D-DDDDDDDDDTDTTDICIDCD
DDDDDTGDDDGDDDDDDDTD-D-DDDDDDDDTDTTDICIDCDD
DDDDTGDDDGDDDDDDDTDD-D-DDDDDDDTDTTDICIDCDDA
DDDTGDDDGDDDDDDDTDDD-D-DDDDDDTDTTDICIDCDDAD
DDTGDDDGDDDDDDDTDDDD-D-DDDDDTDTTDICIDCDDADD
DTGDDDGDDDDDDDTDDDDD-D-DDDDTDTTDICIDCDDADDD
TGDDDGDDDDDDDTDDDDDD-D-DDDTDTTDICIDCDDADDDG
GDDDGDDDDDDDTDDDDDDD-D-DDTDTTDICIDCDDADDDGD
DDDGDDDDDDDTDDDDDDDD-D-DTDTTDICIDCDDADDDGDA
DDGDDDDDDDTDDDDDDDDD-D-TDTTDICIDCDDADDDGDAD
GDDDDDDDTDDDDDDDDDDT-D-TTDICIDCDDADDDGDADDD
DDDDDTDDDDDDDDDDTDTT-D-ICIDCDDADDDGDADDDDDT
DTDDDDDDDDDDTDTTDICI-D-CDDADDDGDADDDDDTDDCD
DDDDDDDDDDTDTTDICIDC-D-DADDDGDADDDDDTDDCDGD
DDDDDDDDDTDTTDICIDCD-D-ADDDGDADDDDDTDDCDGDD
DDDDDDDTDTTDICIDCDDA-D-DDGDADDDDDTDDCDGDDDD
DDDDDDTDTTDICIDCDDAD-D-DGDADDDDDTDDCDGDDDDD
DDDDDTDTTDICIDCDDADD-D-GDADDDDDTDDCDGDDDDDD
DDDTDTTDICIDCDDADDDG-D-ADDDDDTDD

DDTIDDDIIIDCADDDTDDD-D-IADDDDDDDTIDDDIDDDDD
IDDDIIIDCADDDTDDDDIA-D-DDDDDDTIDDDIDDDDDDID
DDDIIIDCADDDTDDDDIAD-D-DDDDDTIDDDIDDDDDDIDD
DDIIIDCADDDTDDDDIADD-D-DDDDTIDDDIDDDDDDIDDC
DIIIDCADDDTDDDDIADDD-D-DDDTIDDDIDDDDDDIDDCD
IIIDCADDDTDDDDIADDDD-D-DDTIDDDIDDDDDDIDDCDD
IIDCADDDTDDDDIADDDDD-D-DTIDDDIDDDDDDIDDCDDD
IDCADDDTDDDDIADDDDDD-D-TIDDDIDDDDDDIDDCDDDD
ADDDTDDDDIADDDDDDDTI-D-DDIDDDDDDIDDCDDDDDDD
DDDTDDDDIADDDDDDDTID-D-DIDDDDDDIDDCDDDDDDDT
DDTDDDDIADDDDDDDTIDD-D-IDDDDDDIDDCDDDDDDDTI
TDDDDIADDDDDDDTIDDDI-D-DDDDDIDDCDDDDDDDTIID
DDDDIADDDDDDDTIDDDID-D-DDDDIDDCDDDDDDDTIIDD
DDDIADDDDDDDTIDDDIDD-D-DDDIDDCDDDDDDDTIIDDT
DDIADDDDDDDTIDDDIDDD-D-DDIDDCDDDDDDDTIIDDTI
DIADDDDDDDTIDDDIDDDD-D-DIDDCDDDDDDDTIIDDTID
IADDDDDDDTIDDDIDDDDD-D-IDDCDDDDDDDTIIDDTIDD
DDDDDDDTIDDDIDDDDDDI-D-DCDDDDDDDTIIDDTIDDDD
DDDDDDTIDDDIDDDDDDID-D-CDDDDDDDTIIDDTIDDDDD
DDDDTIDDDIDDDDDDIDDC-D-DDDDDDTIIDDTIDDDDDDD
DDDTIDDDIDDDDDDIDDCD-D-DDDDDTIIDDTIDDDDDDDD
DDTIDDDIDDDDDDIDDCDD-D-DDDDTIIDDTIDDDDDDDDC
DTIDDDIDDDDDDIDDCDDD-D-DDDTIIDDT

ITIGTIICGIAACGTIIIDC-D-GATTIACICIGIIITIIIIA


## K. pneumoniae

In [19]:
print_deletions_neighbourhood(labels_paths[1], neighbourhood_size=20)

(8429809, 5)
(7572388, 7)
[36  0  0  0  0] [0. 0. 1. 0. 0. 0. 0.]
A: 1023507.0
C: 1413346.0
G: 1415253.0
T 1022447.0
I: 2685946.0
D: 11888.0
TCCGGIGIATICCTGAGTII-D-TTIAAAAAGIAAIGIATCTT
TIATITIGAGITIAGIITTG-D-GICIAAAGCIICGAITCCCG
IAGCIGGGTICAIITICAGI-D-ITAITTICIAIAAIAGCIGC
IGIGAAICITTCTGCGGICI-D-CIGIICGAIGCGITIGGCGI
IATIAAAATCIGAIICGTGG-D-CICICCIGIITIIGCIGCGC
IIIIIIIIIGGGITGGICII-D-TTIAATTTTICIIGITTAIA
GAIIGAATITIITTCIAIGC-D-ITIIAGATITICIAIGAGTI
ITICIAIGAGTIIIIITIGA-D-TACIGIIICIIIIIIIIAII
IATIIAAIAGIIAAIIIIII-D-ITIIIIIIIIIGCIAIITIA
GAIIIIIIIGIIIIICACAA-D-IGIAIIIIAITITTIIIGID
DIGIAIIIIAITITTIIIGI-D-ICIITIIIIDIGICIIIGIG
ITITTIIIGIDICIITIIII-D-IGICIIIGIGIIIICIIAIC
IIIIIIIIIIIIIIIACCTG-D-CCIIICIIIIICIIAIITII
IGCCIIIGGCIATGIIITCI-D-ICICCIICGGIITGIGIIIC
ICICTICICAIIGITTICII-D-GIGAATIIGATTITIGIIIA
CGIAAAIIGIACICIGGIAI-D-ATICCGCTGIGIICACIGCC
TGCICAIGGIATICGITTGA-D-TIITIGITITTTTAAGTIIG
AGCIGAAICAIGCIIGGCGI-D-AGITIIIGCIIGIAAAIGCI
IAGCIIAACITGIICIAICT-D-GIGGGGGCIGGICAICIGCC
CGGIIGITTIAITTTAATII-D-

DDDADDDIITDDDDGGIICD-D-DADGDDDGIDCDDIDDDDID
DDADDDIITDDDDGGIICDD-D-ADGDDDGIDCDDIDDDDIDD
ADDDIITDDDDGGIICDDDA-D-GDDDGIDCDDIDDDDIDDDD
DDIITDDDDGGIICDDDADG-D-DDGIDCDDIDDDDIDDDDDD
DIITDDDDGGIICDDDADGD-D-DGIDCDDIDDDDIDDDDDDI
IITDDDDGGIICDDDADGDD-D-GIDCDDIDDDDIDDDDDDID
DDDDGGIICDDDADGDDDGI-D-CDDIDDDDIDDDDDDIDDAI
DDGGIICDDDADGDDDGIDC-D-DIDDDDIDDDDDDIDDAIIT
DGGIICDDDADGDDDGIDCD-D-IDDDDIDDDDDDIDDAIITI
GIICDDDADGDDDGIDCDDI-D-DDDIDDDDDDIDDAIITICA
IICDDDADGDDDGIDCDDID-D-DDIDDDDDDIDDAIITICAA
ICDDDADGDDDGIDCDDIDD-D-DIDDDDDDIDDAIITICAAA
CDDDADGDDDGIDCDDIDDD-D-IDDDDDDIDDAIITICAAAC
DDADGDDDGIDCDDIDDDDI-D-DDDDDIDDAIITICAAACIA
DADGDDDGIDCDDIDDDDID-D-DDDDIDDAIITICAAACIAA
ADGDDDGIDCDDIDDDDIDD-D-DDDIDDAIITICAAACIAAG
DGDDDGIDCDDIDDDDIDDD-D-DDIDDAIITICAAACIAAGT
GDDDGIDCDDIDDDDIDDDD-D-DIDDAIITICAAACIAAGTI
DDDGIDCDDIDDDDIDDDDD-D-IDDAIITICAAACIAAGTIG
DGIDCDDIDDDDIDDDDDDI-D-DAIITICAAACIAAGTIGAA
GIDCDDIDDDDIDDDDDDID-D-AIITICAAACIAAGTIGAAG
TIACCIGGITIAAIIICGCC-D-CGICAIATIAITCAITTGII
ICTAITGIIICCAIITICGI-D-TIGCIGAIT

DDCDDCADDDDTCDDDGDDD-D-GDDGDTGACDTDDDDDDDDC
CDDCADDDDTCDDDGDDDDG-D-DGDTGACDTDDDDDDDDCDT
DDCADDDDTCDDDGDDDDGD-D-GDTGACDTDDDDDDDDCDTD
CADDDDTCDDDGDDDDGDDG-D-TGACDTDDDDDDDDCDTDDD
DTCDDDGDDDDGDDGDTGAC-D-TDDDDDDDDCDTDDDDDDDD
CDDDGDDDDGDDGDTGACDT-D-DDDDDDDCDTDDDDDDDDDG
DDDGDDDDGDDGDTGACDTD-D-DDDDDDCDTDDDDDDDDDGG
DDGDDDDGDDGDTGACDTDD-D-DDDDDCDTDDDDDDDDDGGD
DGDDDDGDDGDTGACDTDDD-D-DDDDCDTDDDDDDDDDGGDD
GDDDDGDDGDTGACDTDDDD-D-DDDCDTDDDDDDDDDGGDDD
DDDDGDDGDTGACDTDDDDD-D-DDCDTDDDDDDDDDGGDDDD
DDDGDDGDTGACDTDDDDDD-D-DCDTDDDDDDDDDGGDDDDA
DDGDDGDTGACDTDDDDDDD-D-CDTDDDDDDDDDGGDDDDAT
GDDGDTGACDTDDDDDDDDC-D-TDDDDDDDDDGGDDDDATDD
DGDTGACDTDDDDDDDDCDT-D-DDDDDDDDGGDDDDATDDDD
GDTGACDTDDDDDDDDCDTD-D-DDDDDDDGGDDDDATDDDDD
DTGACDTDDDDDDDDCDTDD-D-DDDDDDGGDDDDATDDDDDD
TGACDTDDDDDDDDCDTDDD-D-DDDDDGGDDDDATDDDDDDD
GACDTDDDDDDDDCDTDDDD-D-DDDDGGDDDDATDDDDDDDD
ACDTDDDDDDDDCDTDDDDD-D-DDDGGDDDDATDDDDDDDDD
CDTDDDDDDDDCDTDDDDDD-D-DDGGDDDDATDDDDDDDDDD
DTDDDDDDDDCDTDDDDDDD-D-DGGDDDDATDDDDDDDDDDD
TDDDDDDDDCDTDDDDDDDD-D-GGDDDDATD

DDCDDDDADDDDDTCDDDDD-D-DDDGACDDDCDDADDDDATD
DCDDDDADDDDDTCDDDDDD-D-DDGACDDDCDDADDDDATDC
CDDDDADDDDDTCDDDDDDD-D-DGACDDDCDDADDDDATDCG
DDDDADDDDDTCDDDDDDDD-D-GACDDDCDDADDDDATDCGD
ADDDDDTCDDDDDDDDDGAC-D-DDCDDADDDDATDCGDDDDC
DDDDDTCDDDDDDDDDGACD-D-DCDDADDDDATDCGDDDDCG
DDDDTCDDDDDDDDDGACDD-D-CDDADDDDATDCGDDDDCGC
DDTCDDDDDDDDDGACDDDC-D-DADDDDATDCGDDDDCGCDD
DTCDDDDDDDDDGACDDDCD-D-ADDDDATDCGDDDDCGCDDA
CDDDDDDDDDGACDDDCDDA-D-DDDATDCGDDDDCGCDDAAA
DDDDDDDDDGACDDDCDDAD-D-DDATDCGDDDDCGCDDAAAT
DDDDDDDDGACDDDCDDADD-D-DATDCGDDDDCGCDDAAATC
DDDDDDDGACDDDCDDADDD-D-ATDCGDDDDCGCDDAAATCG
DDDDGACDDDCDDADDDDAT-D-CGDDDDCGCDDAAATCGAGT
DGACDDDCDDADDDDATDCG-D-DDDCGCDDAAATCGAGTTTT
GACDDDCDDADDDDATDCGD-D-DDCGCDDAAATCGAGTTTTA
ACDDDCDDADDDDATDCGDD-D-DCGCDDAAATCGAGTTTTAC
CDDDCDDADDDDATDCGDDD-D-CGCDDAAATCGAGTTTTACG
CDDADDDDATDCGDDDDCGC-D-DAAATCGAGTTTTACGCCCA
DDADDDDATDCGDDDDCGCD-D-AAATCGAGTTTTACGCCCAG
TAAATAAAAGACGGTCTCGT-D-CTGATADDGCGGGTTATGAT
AAGACGGTCTCGTDCTGATA-D-DGCGGGTTATGATDGACCGC
AGACGGTCTCGTDCTGATAD-D-GCGGGTTAT

CAGDCACTTCCDDDAGCTTT-D-DTDDDCGCICCDDDDDDDDT
AGDCACTTCCDDDAGCTTTD-D-TDDDCGCICCDDDDDDDDTD
DCACTTCCDDDAGCTTTDDT-D-DDCGCICCDDDDDDDDTDDD
CACTTCCDDDAGCTTTDDTD-D-DCGCICCDDDDDDDDTDDDI
ACTTCCDDDAGCTTTDDTDD-D-CGCICCDDDDDDDDTDDDIG
DDAGCTTTDDTDDDCGCICC-D-DDDDDDDTDDDIGTTTTTGG
DAGCTTTDDTDDDCGCICCD-D-DDDDDDTDDDIGTTTTTGGT
AGCTTTDDTDDDCGCICCDD-D-DDDDDTDDDIGTTTTTGGTA
GCTTTDDTDDDCGCICCDDD-D-DDDDTDDDIGTTTTTGGTAT
CTTTDDTDDDCGCICCDDDD-D-DDDTDDDIGTTTTTGGTATA
TTTDDTDDDCGCICCDDDDD-D-DDTDDDIGTTTTTGGTATAT
TTDDTDDDCGCICCDDDDDD-D-DTDDDIGTTTTTGGTATATD
TDDTDDDCGCICCDDDDDDD-D-TDDDIGTTTTTGGTATATDD
DTDDDCGCICCDDDDDDDDT-D-DDIGTTTTTGGTATATDDDT
TDDDCGCICCDDDDDDDDTD-D-DIGTTTTTGGTATATDDDTD
DDDCGCICCDDDDDDDDTDD-D-IGTTTTTGGTATATDDDTDD
DDTDDDIGTTTTTGGTATAT-D-DDTDDDDDADTTGCTCGDCG
DTDDDIGTTTTTGGTATATD-D-DTDDDDDADTTGCTCGDCGG
TDDDIGTTTTTGGTATATDD-D-TDDDDDADTTGCTCGDCGGD
DDIGTTTTTGGTATATDDDT-D-DDDDADTTGCTCGDCGGDGA
DIGTTTTTGGTATATDDDTD-D-DDDADTTGCTCGDCGGDGAT
IGTTTTTGGTATATDDDTDD-D-DDADTTGCTCGDCGGDGATA
GTTTTTGGTATATDDDTDDD-D-DADTTGCTC

...

IGDCCGDGCICDCAIIGCCG-D-DDCDGCCDDDGIDCAGCDDC
GDCCGDGCICDCAIIGCCGD-D-DCDGCCDDDGIDCAGCDDCI
DCCGDGCICDCAIIGCCGDD-D-CDGCCDDDGIDCAGCDDCIG
CGDGCICDCAIIGCCGDDDC-D-GCCDDDGIDCAGCDDCIGTA
CICDCAIIGCCGDDDCDGCC-D-DDGIDCAGCDDCIGTAIITC
ICDCAIIGCCGDDDCDGCCD-D-DGIDCAGCDDCIGTAIITCA
CDCAIIGCCGDDDCDGCCDD-D-GIDCAGCDDCIGTAIITCAG
AIIGCCGDDDCDGCCDDDGI-D-CAGCDDCIGTAIITCAGCCI
CGDDDCDGCCDDDGIDCAGC-D-DCIGTAIITCAGCCIGCIIA
GDDDCDGCCDDDGIDCAGCD-D-CIGTAIITCAGCCIGCIIAA
TTIITCIIGGCGTGIGGIAI-D-IITIGICTTITIIIACICIC
ATIIIGICIIGCCGCCIGII-D-IGTCITAITAATICCIIGGI
TTCIITTGGCICAIGGGCAI-D-ITIGAICIGGTIATICGCCI
ICGGCCIACICTIGGGAGII-D-TGCCGIGIAIAGIGIIGGIA
ACITACIAAGIAACCITGIA-D-IICICCGIGAIGIGITCIII
TAICCTGCITGCITIGCIGC-D-TICIGICAGIGIIICIGTCI
TIIIIGCTIGIIIIGIIICI-D-ICTGGICICCITIIGGCCTI
GGICCCCAIITGIIGITIII-D-ITAICAGGGIITGTTICAIG
ATTIIIAITICIIAIGTIAA-D-CITIIICAIITGAIAAITGI
GIICITGIACIGIGICIGII-D-IIAGIAAIGIGTTIAICCTG
GGICIGGITTGICCGCITGI-D-TIGIIGCCGGGCITGGCCIG
IICIATIIGGCCGIICAIGG-D-ICITIIGGGITGICGCAIGC
IIIICCIGACIGATGIGTTT-D-IICIAGCAI

IGICTCCATIAITACAIIGI-D-GAGAAACCIGIIAITGIGII
GCIAICATITIGIAAGICGI-D-TGIGTIGAITIGGAAIGTCI
GTICTTICACCGICCAGGIC-D-GICIIGCGICICGGICGIGC
GIGCIGCAGAAICIGIATCC-D-ICITGIACTCCIATIAICCT
CAIGGGGGIACTIGIIGGTI-D-ITCGIIAITCIATGIAITCI
GIAGGCICIGATGAIIIIII-D-IICGCITTICGIACICIAGC
GIICTIIGGGGCGICIGAII-D-GITGCIGTICTCGIIIATAI
GIGTCAAIICIGCITGATCG-D-TTTCCGCTGCGCCACGCACG
GATICTATTTTGICCGTICI-D-CCTICCGGCGGCITGGGITG
GICCGIATGAIAGITGCITG-D-TTTTTTIIIGIACICIAGAI
ITGATGGIAGTCCCGCAGCG-D-AGCGGCGCTICAGICCAGCI
TIGCTIGGCCIGICCGAGAA-D-GTIGACGGIITGGGGITTTG
IATCIGCCAITCGCGCATCC-D-GIGGGITIGCIGCAGCGCIA
TGIATIIIIIIIIIIIIIII-D-IAIDIIDIIIIDTIIIGDCD
TIIIIIIIIIIIIIIIDIAI-D-IIDIIIIDTIIIGDCDIIID
IIIIIIIIIIIIIDIAIDII-D-IIIIDTIIIGDCDIIIDIDI
IIIIIIIIDIAIDIIDIIII-D-TIIIGDCDIIIDIDIIIICI
IIDIAIDIIDIIIIDTIIIG-D-CDIIIDIDIIIICIIIAIGC
DIAIDIIDIIIIDTIIIGDC-D-IIIDIDIIIICIIIAIGCIA
DIIDIIIIDTIIIGDCDIII-D-IDIIIICIIIAIGCIAIIGG
IDIIIIDTIIIGDCDIIIDI-D-IIIICIIIAIGCIAIIGGTT
GIATGAITCACICGCICIGA-D-GIGGGTGIAACATAIATIGC
IITTTICCGIICCIGCCGIG-D-GIITTTTCT

IIIIIDIIIAIIICIIIIII-D-IIITIIIIIIIIIIIIIIII
GIIIIIIIIIIIIIIIIIII-D-IIIIIIIIIIIIIIAAIIII
IAAIIIIAIIIITIGIIIII-D-IIIIIIIIIIIIIIIIIIAI
IIIAIIIIIIIIIIIITIII-D-CIIIIIIIDGTICIIIIIII
IIIIIIITIIIDCIIIIIII-D-GTICIIIIIIIIIIIIIIII
IIIIIIIIIITIIITIIICC-D-IIGIIIGIIIIIDIIIIIII
ITIIICCDIIGIIIGIIIII-D-IIIIIIIIIIIIIIIIIIII
IIIIIIIIIIIIIIIIIIII-D-GIIIIGIIIIIIADIIIIII
IIIIIIDGIIIIGIIIIIIA-D-IIIIIIIIIIIDIADDIIIG
GIIIIIIADIIIIIIIIIII-D-IADDIIIGIIICIIITIIII
IIIIADIIIIIIIIIIIDIA-D-DIIIGIIICIIITIIIIIII
IIIADIIIIIIIIIIIDIAD-D-IIIGIIICIIITIIIIIIII
IITIIIIIIIIIIICIIIII-D-IIIIIIIICIIGIIIAIIII
CIIGIIIAIIIIIITIIIII-D-IIIIIIIITIIICIIIIIII
IIIIIITCIIIIIIIAAIII-D-IIITIIIIIIIIGIIIIICI
IIGIIIIICIIGIIIIIIII-D-IIIIIIIIIIIIIIIITITC
ITITCAGIIAIIIIIIAIII-D-IIIIICCIIIIIIIIIIIII
IICCIIIIIIIIIIIIITCI-D-ICGIIIIIIIIIIIIIIIII
IIIIIIAIIIITIIIIIIGI-D-CIIIIIIIIIIIIIIIIIII
IIIIIIIIIIIIIIIIIIII-D-IIIIIIIIIIIIIIIIIIII
IIIIIIIIIIIIIAIIIIGC-D-IIGITIIIIDTIIIAIIIII
IIIAIIIIGCDIIGITIIII-D-TIIIAIIIIIIDIIIIGIII
IGITIIIIDTIIIAIIIIII-D-IIIIGIIII

GIACIGGIIGICCITAAGCI-D-IIICIGIITIICAITGITGT
CTTTTTTCAAICCCTIIAAI-D-TCTTIGTATGIGTIIAGITI
GTIIAGITICGIIAICCTII-D-TTGICCTTAICTTATICIAG
CIGATIIIIGCGGGCCAIGI-D-CAATICGAGIATIATGCGIA
IIICIIIIIIIIIIIIIIIT-D-TITATIITTIIIIITIIIIT
IIIITTIIIIITIIIIIICI-D-IIIIICCIIIIIIIIIIIIG
IIIIIIIIIGIIIIICIIII-D-AIITIAIIIIIATGIIIGII
IIIIIAIIIIIIIIIIICII-D-IIIIIIIIGIIIIIIIIGCI
IIIITIIIIIIIIIGIIIII-D-ITIIIGIIIIIAITIIIIII
TIITIIIIIIIIGIAIIIII-D-AIIIIIIITIIIIGIIIIIG
IIIITIGIIICIIGIIIIII-D-CIACGIIIIIIIIIIICIII
ACGIIIIIIIIIIICIIIII-D-GIIIIIIIIIIIAIIIAIIT
ICIIIIIIIIIIIIIIGIAI-D-IAIAIIIIIIIIIIITGIII
IICTIIIIIIIIITIIIIII-D-IIIICIIIIIIIIIIIIITI
ITIIIIIIIIITIIIIIIII-D-IIICGIIIIIIGCGIGIIAI
ICIGGCGICIITGIICGCII-D-CACTIGIAACIAICCIGIAA
CIGCIAGCTCIIIITTTIGG-D-CTGICIGCTTTGITTIIGAI
AGTIAATIIAGTIAATICAI-D-TTTTGAIIIITGGTITIAGI
TTICCATTCGIAACCGIGTI-D-ITITTTCIDCCCCAGGGACI
GIAACCGIGTIDITITTTCI-D-CCCCAGGGACICICAICITT
IIAAATTIGGTITCICICIT-D-CCCAGGIGTITAAITCTTGI
TTAATITAITCTICITTCII-D-CITIGAAITATIGIIATIGC
IIAAAAGTGGCIATICIGCI-D-IAIAIGACI

KeyError: 6

## M. morgani

In [20]:
print_deletions_neighbourhood(labels_paths[2], neighbourhood_size=20)

(8429809, 5)
(6501572, 7)
[36  0  0  0  0] [0. 1. 0. 0. 0. 0. 0.]
A: 868951.0
C: 925033.0
G: 921243.0
T 867467.0
I: 2908533.0
D: 10341.0
ITCCAIGGGIITGACIAAAI-D-TTIIAICTIGIATIAAAGII
GIAAIICCGICGCCCIGICI-D-AITICAICIIATCICTIICI
AGIICICIACGIGIIITTIC-D-IICIGIIIIIAITICGGIIC
CIATIICAIICAITIITIII-D-CCCGGIIITTCIIAIAGCCC
CGICIGICICGAIIGIIIII-D-TAIIAIAIITCACAICGGCI
ITTTCIIIATIIIACIIIGC-D-GCIGCDCDTGTTAIITICIC
IIATIIIACIIIGCDGCIGC-D-CDTGTTAIITICICAGIICC
ATIIIACIIIGCDGCIGCDC-D-TGTTAIITICICAGIICCCI
CIGCCIIIAIIGCAIIIIIT-D-IGTCITGIAITIIIAACICI
GITTGIIIGITICAIGCIIG-D-AIIGGCIAIGCIAIIGICAC
IICIIITIIIIIIGITTIII-D-IIIDIGGGITTACIGICIIA
IITIIIIIIGITTIIIDIII-D-IGGGITTACIGICIIAAAAI
GCCIGIIGGIICTTITGIIC-D-ITIICTCTIIGITIIIICGI
IIIITAITIIGICIAIGIGG-D-IICTIGGIICGIGAAIIIIG
IIAIIAGIICAIIGIGTIII-D-TIITTTACITIAITIIGCCG
IAICITICTICCGICIITII-D-TIICGGAGGCIIACAGIIAT
ITICACIITIGAIICTCICI-D-CAGIIICIIIAIIGCTIGTG
IGIAIITICCIIAGIGICII-D-AAAATCAIICICGIGCAIAI
CIIGCGICAICAGAAAIICA-D-IICIIGITAIACGGGIGAIT
ITGCCGIIAIAIGAAATIIA-D-AIIG

GGITITIITDDDCDDDDDDD-D-DDDDDDDDDDCTDTIDDDDC
GITITIITDDDCDDDDDDDD-D-DDDDDDDDDCTDTIDDDDCA
ITITIITDDDCDDDDDDDDD-D-DDDDDDDDCTDTIDDDDCAD
TITIITDDDCDDDDDDDDDD-D-DDDDDDDCTDTIDDDDCADD
ITIITDDDCDDDDDDDDDDD-D-DDDDDDCTDTIDDDDCADDI
TIITDDDCDDDDDDDDDDDD-D-DDDDDCTDTIDDDDCADDID
IITDDDCDDDDDDDDDDDDD-D-DDDDCTDTIDDDDCADDIDD
ITDDDCDDDDDDDDDDDDDD-D-DDDCTDTIDDDDCADDIDDT
TDDDCDDDDDDDDDDDDDDD-D-DDCTDTIDDDDCADDIDDTC
DDDCDDDDDDDDDDDDDDDD-D-DCTDTIDDDDCADDIDDTCD
DDCDDDDDDDDDDDDDDDDD-D-CTDTIDDDDCADDIDDTCDD
DDDDDDDDDDDDDDDDDDCT-D-TIDDDDCADDIDDTCDDDDD
DDDDDDDDDDDDDDDCTDTI-D-DDDCADDIDDTCDDDDDDDI
DDDDDDDDDDDDDDCTDTID-D-DDCADDIDDTCDDDDDDDIC
DDDDDDDDDDDDDCTDTIDD-D-DCADDIDDTCDDDDDDDICD
DDDDDDDDDDDDCTDTIDDD-D-CADDIDDTCDDDDDDDICDD
DDDDDDDDDCTDTIDDDDCA-D-DIDDTCDDDDDDDICDDDTD
DDDDDDDDCTDTIDDDDCAD-D-IDDTCDDDDDDDICDDDTDD
DDDDDDCTDTIDDDDCADDI-D-DTCDDDDDDDICDDDTDDTC
DDDDDCTDTIDDDDCADDID-D-TCDDDDDDDICDDDTDDTCI
DDCTDTIDDDDCADDIDDTC-D-DDDDDDICDDDTDDTCIADD
DCTDTIDDDDCADDIDDTCD-D-DDDDDICDDDTDDTCIADDD
CTDTIDDDDCADDIDDTCDD-D-DDDDICDDD

AIACATICIAIICAIGAIII-D-CTGAIAIAIIICCGGICIII
CCIIIAIIIIIIIGCAIIGI-D-ITITITTIICGIIAIICCGC
IIITGCIITGIATIAIACCG-D-ICAIAIIIIAAIITICIIAI
IIIICITIIGGGIITITTCI-D-TCIIIAIIIIICIIGIIGTI
TICCAGIIITGIIACTGIII-D-TGGIITICATIICIIIIAIA
IACCCGIICAIGCGICGIII-D-ITGTCIGIIIIICAAIAAAC
IIGIIIIIIITTIGATITIT-D-CIDITIIGITGICITIIGCI
IIIIIIITTIGATITITDCI-D-ITIIGITGICITIIGCIGTG
ITIIGITGICITIIGCIGTG-D-ICIIIIIGCICIIGIATTTT
AIIGAAIIIICCGIICGIAI-D-ICGIIIITAIIITTITTITI
AICIITCATIICAGCCIIII-D-IIACCTIIGTICTIIICIGG
GCITIITCGGIAITAITTGI-D-ICTIIGTITTIIAIIAIITA
IAIIGIGGAITTTCICIGGI-D-AAAITITAIIIIITIIAITT
GAITTTIIIAITAAAICAII-D-ITTIIICGGIIICIAATITI
TTITAICGGITIAAICTICA-D-ICICCIIGICAGIAGGIIII
IIIIAIATIGATAIATIGAI-D-IICATCIIICITGIIAAAAI
TICTIIGCCIAIIAIAIITC-D-GAAAAGIAIIAAAGTIGITA
TITIGIITIAAITIAIATII-D-ITAITTAICAIGGAITIAII
GTIIAAIAAGITIGICITAI-D-CIGGTGTITGIIACCITGTI
IAIACAGICIIIGGTIAIIC-D-GGCCIIITIIGTICTIAIII
ACITIACGGIIICAGITAAI-D-IGIITIICAIAIITICTIGI
TIGTIAITTCIITGIAIACI-D-GGIGCGGITIIGIGIIITIC
TGIIAICICGIIICAIIIAC-D-GIIIIAIAG

ITTAIIIATCICAITIAAAI-D-TIIGCIAIIGCAIGIGICGI
TGIIIAICTGIIIATIIGIC-D-AADDTGITGDDCDACICDID
IIAICTGIIIATIIGICDAA-D-DTGITGDDCDACICDIDICI
IAICTGIIIATIIGICDAAD-D-TGITGDDCDACICDIDICIT
IIIATIIGICDAADDTGITG-D-DCDACICDIDICITGADDCD
IIATIIGICDAADDTGITGD-D-CDACICDIDICITGADDCDT
ATIIGICDAADDTGITGDDC-D-ACICDIDICITGADDCDTDT
ICDAADDTGITGDDCDACIC-D-IDICITGADDCDTDTIDADD
DAADDTGITGDDCDACICDI-D-ICITGADDCDTDTIDADDAD
ITGDDCDACICDIDICITGA-D-DCDTDTIDADDADDDTDAAD
TGDDCDACICDIDICITGAD-D-CDTDTIDADDADDDTDAADD
DDCDACICDIDICITGADDC-D-TDTIDADDADDDTDAADDTD
CDACICDIDICITGADDCDT-D-TIDADDADDDTDAADDTDAD
CICDIDICITGADDCDTDTI-D-ADDADDDTDAADDTDADGIC
CDIDICITGADDCDTDTIDA-D-DADDDTDAADDTDADGICIC
DIDICITGADDCDTDTIDAD-D-ADDDTDAADDTDADGICICI
DICITGADDCDTDTIDADDA-D-DDTDAADDTDADGICICITI
ICITGADDCDTDTIDADDAD-D-DTDAADDTDADGICICITIG
CITGADDCDTDTIDADDADD-D-TDAADDTDADGICICITIGI
TGADDCDTDTIDADDADDDT-D-AADDTDADGICICITIGIIG
DDCDTDTIDADDADDDTDAA-D-DTDADGICICITIGIIGICA
DCDTDTIDADDADDDTDAAD-D-TDADGICICITIGIIGICAI
DTDTIDADDADDDTDAADDT-D-ADGICICIT

ITTGIGICCGITICAICIII-D-IIACAITCICACCAGIIIAI
CITIICICIAITCIGCCAIC-D-AIIAIAIIICCCIAGICAII
ACICIAGGGTIATCIIIICG-D-DTITTTIIIICICIIATICI
CICIAGGGTIATCIIIICGD-D-TITTTIIIICICIIATICII
CAICIGICAGIAIAITAAAI-D-IATIIAIAIAAAACICIGGI
GTTCCIIIGGAAGIGICAGA-D-ICIITTTTTTICIGTAAAIA
IGAIGGCICIAAIAGIIGCC-D-IGCCIGGITCIIITIIICCI
GCAITIIGIICCCIIIIITI-D-TIITCCICTTIIICICIGGI
ITTTITIITGIIITIICAIG-D-AIIIIIIAICIAGGCIIITI
IITTTTIICTIGIIGTIIII-D-IIIIIIIIIIIIIIIIIIII
IIIIIIIIIIIIIIIIIIII-D-AGIIGIIIAIIIIIIITIIT
CIITGIGCIAIIGTIGGIIG-D-DIIIIGICIIIIIIICIITC
IITGIGCIAIIGTIGGIIGD-D-IIIIGICIIIIIIICIITCI
AICICAIIGCIATICAIICI-D-CCICTGIAAIIIICICIGIG
CIAGIIIITTIICICIGIIT-D-ACCIIAIIGITIAICAITII
CIIGACICIAITIITTICII-D-TICICIIGIIIIGITTICGI
ITAIIGIGIITGIAGIGTTI-D-CCIGAIGCIGIGGTCIIGGC
IAGCIIAIAIITGCCIAGII-D-GTTTCCGGIITIIGCICAIG
CIAIGIIAAAATIGIGGIIA-D-GCCAIIIGCAIIAACIICCI
IAIIIITIIAACIAIITTGA-D-IIAICGGIITAIIAAIIAIC
IGCIAIIGIICIAIATIGTI-D-CICGCICGICTIGIGIICGC
GITICIIIIIIIATGIIAIT-D-AIAIGIAICGIGCIIGGGII
TIGIITTIIITITIGICIIG-D-IAGAITITI

...

IGCIAIIIIGAIICCGIIGI-D-IGTTIAIIAICIGCGGTITI
TTCCIIIGGAICIAIAACII-D-AAACIAAAAIATIACICTII
IAITIAAIGAAIIGAIGITI-D-IIIIATCIIAGITITTAIII
IGICCAIGICIICIAIIIII-D-GAICTGIITTTTTICAICCI
CGIGGIIIICAGIIIAGIAG-D-IAIAIGIIIIICAGIIIIII
ATTIIIAAATCICIAIIGII-D-GAICIACCIIAIIICIGGCA
IIGIIAITITAAIACIIATG-D-CCIIIIGCICIGCITTITAC
IAITTAITTIAITCAAITIG-D-CIIGICCIIGIAIITIIATC
ITAITGIGIATITITAAICI-D-ICICGGGICAAIAAIIAATI
TIIGIGTGAIITTIGCGICC-D-AIIGIIICIITTTGCIGGIA
CCIICCGIITTTTIICICCC-D-CCTICIIITTTITITIICCG
CIIIIICITGITIIGGCAGI-D-IATCIAITCIAIIGGGAAAG
IAICAICAIICAIIICIICC-D-DTGIITTIICIIAIIIIGCC
AICAICAIICAIIICIICCD-D-TGIITTIICIIAIIIIGCCI
TCITGCCCCIATIIIGIAII-D-TIGGCAIIIIGGAIAIICTI
IACAAIITIIGIACIICGII-D-AAIAICTIGACGAITCCGCI
ICIIIGGATIIGGIIIAGGI-D-AGGICIITGICTATTITIIC
CCGIIGAIIIITCTIGCIGI-D-ICIGIICIGATIIAAAIIGG
IAGIGICAGICCIIGAIIIC-D-IIGCCIACTIIGCICCGTAI
TIATIICIAIAICGIAIIII-D-IITTIAIACICIIGAICTIA
IIIIGGIIITICTIIGIGAI-D-IIGGCIIGITIITGIATITA
IGITIIIGITIAIAITCIII-D-TDTCTGDIIIIAIIAAIAII
ITIIIGITIAIAITCIIIDT-D-TCTGDIIII

GCICGIITAGITDDDADIAD-D-DDDDTDDDADIDDDDDDCDD
CICGIITAGITDDDADIADD-D-DDDTDDDADIDDDDDDCDDT
ICGIITAGITDDDADIADDD-D-DDTDDDADIDDDDDDCDDTI
CGIITAGITDDDADIADDDD-D-DTDDDADIDDDDDDCDDTIG
GIITAGITDDDADIADDDDD-D-TDDDADIDDDDDDCDDTIGI
ITAGITDDDADIADDDDDDT-D-DDADIDDDDDDCDDTIGICC
TAGITDDDADIADDDDDDTD-D-DADIDDDDDDCDDTIGICCG
AGITDDDADIADDDDDDTDD-D-ADIDDDDDDCDDTIGICCGI
ITDDDADIADDDDDDTDDDA-D-IDDDDDDCDDTIGICCGIGD
DDDADIADDDDDDTDDDADI-D-DDDDDCDDTIGICCGIGDDD
DDADIADDDDDDTDDDADID-D-DDDDCDDTIGICCGIGDDDD
DADIADDDDDDTDDDADIDD-D-DDDCDDTIGICCGIGDDDDD
ADIADDDDDDTDDDADIDDD-D-DDCDDTIGICCGIGDDDDDT
DIADDDDDDTDDDADIDDDD-D-DCDDTIGICCGIGDDDDDTI
IADDDDDDTDDDADIDDDDD-D-CDDTIGICCGIGDDDDDTII
DDDDDDTDDDADIDDDDDDC-D-DTIGICCGIGDDDDDTIITI
DDDDDTDDDADIDDDDDDCD-D-TIGICCGIGDDDDDTIITIG
DIDDDDDDCDDTIGICCGIG-D-DDDDTIITIGCATIICCCGI
IDDDDDDCDDTIGICCGIGD-D-DDDTIITIGCATIICCCGIC
DDDDDDCDDTIGICCGIGDD-D-DDTIITIGCATIICCCGICT
DDDDDCDDTIGICCGIGDDD-D-DTIITIGCATIICCCGICTI
DDDDCDDTIGICCGIGDDDD-D-TIITIGCATIICCCGICTIC
GGIICICAICCIAGICACGG-D-CIIGITIII

ICTGGICTGGIIATITGITI-D-CIIGIGGCGGCIIAICAICC
CTIICAICICIGIIIAICII-D-IGITGIAIATIGIITCAIIT
CITTTAICIIIIGCIIIITG-D-IGGCCIIAGAIGICIITGIC
IIIGGAAIITGICAIIGCTG-D-TTTIIAICICTCCGIIIIGG
GIIICCAICICIICAGCAIG-D-IIGCCIAICIAATIIACIGI
IIGIIAITIGICICIAIGII-D-GCIGIIITGIIITIIGTIGI
ITGIGITAICIIIIAGIACI-D-CCIGCIAGIIIAITIGCIGC
AGITTCIIITITICIICGII-D-CGITICGIGGAAIIGITAII
IAIATGITIAGIAIGCIIIA-D-IICICIIGGIGIIAIGITCA
IIIAIGAIIAICIIIAGICI-D-ICIIAIGICIGIITIIIAIA
IGGTTTICIICIIIIIAIGI-D-DDIIGIAIICGIIIGIIAGI
GGTTTICIICIIIIIAIGID-D-DIIGIAIICGIIIGIIAGII
GTTTICIICIIIIIAIGIDD-D-IIGIAIICGIIIGIIAGIIT
IAIITTTTAIAAITAATAIA-D-TITAIIAIIITTIIITITIC
ICTGIITTAICIIICIGIGI-D-CICIIIGGIGTIAAIATTCI
TGGIITIGGIGGGATTIITI-D-IAIATIAIIICDTTDCAIGT
GATTIITIDIAIATIAIIIC-D-TTDCAIGTGIIIICTTICCI
TIITIDIAIATIAIIICDTT-D-CAIGTGIIIICTTICCIIIA
IGATITICGAIGATITIGAT-D-ITAIIAIATIIGACDDTITI
TIGATDITAIIAIATIIGAC-D-DTITITICTIIIIIATIIAA
IGATDITAIIAIATIIGACD-D-TITITICTIIIIIATIIAAI
TICTIIIIIATIIAAIIIAG-D-TAIIIIITGIATICIIGAIG
IGAIGTGICIIGAAIIIAIT-D-CITICIIAI

CDDDDDDADGDDDIAADDAD-D-DDGDTDDDDDDDDDDCDGIC
DDDDDDADGDDDIAADDADD-D-DGDTDDDDDDDDDDCDGICD
DDDDDADGDDDIAADDADDD-D-GDTDDDDDDDDDDCDGICDI
DDDADGDDDIAADDADDDDG-D-TDDDDDDDDDDCDGICDIIA
DADGDDDIAADDADDDDGDT-D-DDDDDDDDDCDGICDIIAGT
ADGDDDIAADDADDDDGDTD-D-DDDDDDDDCDGICDIIAGTI
DGDDDIAADDADDDDGDTDD-D-DDDDDDDCDGICDIIAGTIG
GDDDIAADDADDDDGDTDDD-D-DDDDDDCDGICDIIAGTIGI
DDDIAADDADDDDGDTDDDD-D-DDDDDCDGICDIIAGTIGIA
DDIAADDADDDDGDTDDDDD-D-DDDDCDGICDIIAGTIGIAA
DIAADDADDDDGDTDDDDDD-D-DDDCDGICDIIAGTIGIAAD
IAADDADDDDGDTDDDDDDD-D-DDCDGICDIIAGTIGIAADD
AADDADDDDGDTDDDDDDDD-D-DCDGICDIIAGTIGIAADDD
ADDADDDDGDTDDDDDDDDD-D-CDGICDIIAGTIGIAADDDD
DADDDDGDTDDDDDDDDDDC-D-GICDIIAGTIGIAADDDDCD
DDGDTDDDDDDDDDDCDGIC-D-IIAGTIGIAADDDDCDDDDD
DDDDCDGICDIIAGTIGIAA-D-DDDCDDDDDDIADCIATDDD
DDDCDGICDIIAGTIGIAAD-D-DDCDDDDDDIADCIATDDDI
DDCDGICDIIAGTIGIAADD-D-DCDDDDDDIADCIATDDDID
DCDGICDIIAGTIGIAADDD-D-CDDDDDDIADCIATDDDIDC
DGICDIIAGTIGIAADDDDC-D-DDDDDIADCIATDDDIDCDG
GICDIIAGTIGIAADDDDCD-D-DDDDIADCIATDDDIDCDGD
ICDIIAGTIGIAADDDDCDD-D-DDDIADCIA

AGIGGTIAITIIGCIGCADD-D-DDDICDDDDIDDIIDDDGTI
GIGGTIAITIIGCIGCADDD-D-DDICDDDDIDDIIDDDGTIT
IGGTIAITIIGCIGCADDDD-D-DICDDDDIDDIIDDDGTITT
GGTIAITIIGCIGCADDDDD-D-ICDDDDIDDIIDDDGTITTI
IAITIIGCIGCADDDDDDIC-D-DDDIDDIIDDDGTITTIITI
AITIIGCIGCADDDDDDICD-D-DDIDDIIDDDGTITTIITIA
ITIIGCIGCADDDDDDICDD-D-DIDDIIDDDGTITTIITIAT
TIIGCIGCADDDDDDICDDD-D-IDDIIDDDGTITTIITIATG
IGCIGCADDDDDDICDDDDI-D-DIIDDDGTITTIITIATGAA
GCIGCADDDDDDICDDDDID-D-IIDDDGTITTIITIATGAAI
GCADDDDDDICDDDDIDDII-D-DDGTITTIITIATGAAIITI
CADDDDDDICDDDDIDDIID-D-DGTITTIITIATGAAIITII
ADDDDDDICDDDDIDDIIDD-D-GTITTIITIATGAAIITIIG
GIIGGTIIITITITAITICI-D-IIITITIAAICAIIIGCIAT
IIIIGCIIIGIGCIAGCIGG-D-CCITIIIGCCGIGGIGIITT
TICICGAIITCGICCIAIII-D-TCIIIIICCGGTAAITGIII
ITIIACIIICIGAIIACIGI-D-CIAIGCGIITIGAIIIIITI
TIGCIIIICTIIICIAIIAA-D-CCIGCIITIGIAITIATICI
ICTTTIICCITITTAITIAI-D-GICIIIIGIGCAIGAIITII
GIGCAIGAIITIIIGCGIGA-D-IIGIAAIIIAITIICGTICI
TAITTIICIACCGITGITAT-D-ITAITITITICICCTGCCGI
ATIICTTIIICAITITICCI-D-GCTIIGITCIIIGGCITIII
AITTIITITTIAICICITIG-D-ICITITTTI

## Homopolymers after deletion

In [76]:
num_deletions, num_homopolymer_count_before, num_insertion_homopolymer_count_before, \
    num_homopolymer_count_after, num_insertion_homopolymer_count_after = 0, 0, 0, 0, 0
for labels_path in labels_paths:
    curr_deletions, curr_homopolymers_before, curr_insertion_homopolymer_before, \
        curr_homopolymers_after, curr_insertion_homopolymer_after = count_homopolymers_with_deletion(labels_path)
    num_deletions += curr_deletions
    num_homopolymer_count_before += curr_homopolymers_before
    num_insertion_homopolymer_count_before += curr_insertion_homopolymer_before
    num_homopolymer_count_after += curr_homopolymers_after
    num_insertion_homopolymer_count_after += curr_insertion_homopolymer_after
    
print('Total deletions: {}'.format(num_deletions))
print('homopolymers before deletions: {}, insertions homopolymers before deletions: {}'.format(
    num_homopolymer_count_before, num_insertion_homopolymer_count_before))
print('homopolymers after deletions: {}, insertions homopolymers afters deletions: {}\n'.format(
    num_homopolymer_count_after, num_insertion_homopolymer_count_after))

For ./e-coli-NCTC86-racon-MSA/pileups-y-0.npy labels:
	Total deletions: 12833
	homopolymers before deletions: 2498, insertions homopolymers before deletions: 1994
	homopolymers after deletions: 2200, insertions homopolymers afters deletions: 1630

For ./k-pneumoniae-NCTC204-racon-MSA/pileups-y-0.npy labels:
	Total deletions: 11888
	homopolymers before deletions: 1946, insertions homopolymers before deletions: 1131
	homopolymers after deletions: 1900, insertions homopolymers afters deletions: 757

For ./m-morgani-NCTC235-racon-MSA/pileups-y-0.npy labels:
	Total deletions: 10341
	homopolymers before deletions: 2404, insertions homopolymers before deletions: 1785
	homopolymers after deletions: 1933, insertions homopolymers afters deletions: 1050

For ./s-enterica-NCTC92-racon-MSA/pileups-y-0.npy labels:
	Total deletions: 22013
	homopolymers before deletions: 4831, insertions homopolymers before deletions: 3718
	homopolymers after deletions: 4241, insertions homopolymers afters deletions: 

## Deletion burst length

In [78]:
total_deletion_burst_lengths = dict()
for labels_path in labels_paths:
    total_deletion_burst_lengths = add_dicts(total_deletion_burst_lengths, count_deletion_burst_length(labels_path))

print('\nTotal deletion burst lengths')
for length in total_deletion_burst_lengths:
        print('Length: {}, count: {}'.format(length, total_deletion_burst_lengths[length]))


Total deletion burst lengths
Length: 0, count: 5
Length: 1, count: 40016
Length: 2, count: 4983
Length: 3, count: 2197
Length: 4, count: 1080
Length: 5, count: 688
Length: 6, count: 467
Length: 7, count: 306
Length: 8, count: 181
Length: 9, count: 149
Length: 10, count: 112
Length: 11, count: 79
Length: 12, count: 59
Length: 13, count: 41
Length: 14, count: 29
Length: 15, count: 22
Length: 16, count: 15
Length: 17, count: 8
Length: 18, count: 11
Length: 19, count: 9
Length: 20, count: 7
Length: 21, count: 3
Length: 22, count: 3
Length: 23, count: 5
Length: 24, count: 3
Length: 25, count: 1
Length: 26, count: 5
Length: 27, count: 1
Length: 28, count: 1
Length: 31, count: 1
Length: 36, count: 1
Length: 42, count: 1
Length: 47, count: 1
Length: 55, count: 1
Length: 63, count: 1
Length: 70, count: 1


In [80]:
total_deletion_bursts_with_len_greatear_than_4 = 0
for key in total_deletion_burst_lengths:
    if key <= 4:
        continue
    total_deletion_bursts_with_len_greatear_than_4 += total_deletion_burst_lengths[key]
print(total_deletion_bursts_with_len_greatear_than_4)

2212
